In [5]:
from z3 import *

x = Int('x')
y = Int('y')
n = x + y >= 3
print ("num args: ", n.num_args())
print ("children: ", n.children())
print ("1st child:", n.arg(0))
print ("2nd child:", n.arg(1))
print ("operator: ", n.decl())
print ("op name:  ", n.decl().name())

num args:  2
children:  [x + y, 3]
1st child: x + y
2nd child: 3
operator:  >=
op name:   >=


In [6]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 > 3, x**3 + y < 5)

[x = 1/8, y = 2]


The procedure Real('x') creates the real variable x. Z3Py can represent arbitrarily large integers, rational numbers (like in the example above), and irrational algebraic numbers. An irrational algebraic number is a root of a polynomial with integer coefficients. Internally, Z3 represents all these numbers precisely. The irrational numbers are displayed in decimal notation for making it easy to read the results.

In [7]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 == 3, x**3 == 2)

[x = 1.2599210498?, y = -1.1885280594?]


The procedure set_option is used to configure the Z3 environment. It is used to set global configuration options such as how the result is displayed. The option set_option(precision=30) sets the number of decimal places used when displaying results. The ? mark in 1.2599210498? indicates the output is truncated.

In [10]:
set_option(precision=30)
print ("Solving, and displaying result with 30 decimal places")
solve(x**2 + y**2 == 3, x**3 == 2)

Solving, and displaying result with 30 decimal places
[x = 1.259921049894873164767210607278?,
 y = -1.188528059421316533710369365015?]


The following example demonstrates a common mistake. The expression 3/2 is a Python integer and not a Z3 rational number. The example also shows different ways to create rational numbers in Z3Py. The procedure Q(num, den) creates a Z3 rational where num is the numerator and den is the denominator. The RealVal(1) creates a Z3 real number representing the number 1.

In [11]:
print (1/3)
print (RealVal(1)/3)
print (Q(1,3))

x = Real('x')
print (x + 1/3)
print (x + Q(1,3))
print (x + "1/3")
print (x + 0.25)

0.3333333333333333
1/3
1/3
x + 3333333333333333/10000000000000000
x + 1/3
x + 1/3
x + 1/4


Rational numbers can also be displayed in decimal notation.

In [12]:
x = Real('x')
solve(3*x == 1)

set_option(rational_to_decimal=True)
solve(3*x == 1)

set_option(precision=30)
solve(3*x == 1)

[x = 1/3]
[x = 0.333333333333333333333333333333?]
[x = 0.333333333333333333333333333333?]


A system of constraints may not have a solution. In this case, we say the system is unsatisfiable.

In [13]:
x = Real('x')
solve(x > 4, x < 0)

no solution


Boolean Logic

Z3 supports Boolean operators: And, Or, Not, Implies (implication), If (if-then-else). Bi-implications are represented using equality ==. The following example shows how to solve a simple set of Boolean constraints.

In [16]:
p = Bool('p')
q = Bool('q')
r = Bool('r')
solve(Implies(p, q), r == Not(q), Or(Not(p), r))

[q = False, p = False, r = True]


The Python Boolean constants True and False can be used to build Z3 Boolean expressions.

In [17]:
p = Bool('p')
q = Bool('q')
print (And(p, q, True))
print (simplify(And(p, q, True)))
print (simplify(And(p, False)))

And(p, q, True)
And(p, q)
False


The following example uses a combination of polynomial and Boolean constraints.

In [18]:
p = Bool('p')
x = Real('x')
solve(Or(x < 5, x > 10), Or(p, x**2 == 2), Not(p))

[x = -1.414213562373095048801688724209?, p = False]


Solvers

Z3 provides different solvers. The command solve, used in the previous examples, is implemented using the Z3 solver API. The implementation can be found in the file z3.py in the Z3 distribution. The following example demonstrates the basic Solver API.

In [19]:
x = Int('x')
y = Int('y')

s = Solver()
print( s)

s.add(x > 10, y == x + 2)
print (s)
print ("Solving constraints in the solver s ...")
print (s.check())

print ("Create a new scope...")
s.push()
s.add(y < 11)
print (s)
print ("Solving updated set of constraints...")
print (s.check())

print ("Restoring state...")
s.pop()
print (s)
print ("Solving restored set of constraints...")
print (s.check())

[]
[x > 10, y == x + 2]
Solving constraints in the solver s ...
sat
Create a new scope...
[x > 10, y == x + 2, y < 11]
Solving updated set of constraints...
unsat
Restoring state...
[x > 10, y == x + 2]
Solving restored set of constraints...
sat


The command Solver() creates a general purpose solver. Constraints can be added using the method add. We say the constraints have been asserted in the solver. The method check() solves the asserted constraints. The result is sat (satisfiable) if a solution was found. The result is unsat (unsatisfiable) if no solution exists. We may also say the system of asserted constraints is infeasible. Finally, a solver may fail to solve a system of constraints and unknown is returned.

In some applications, we want to explore several similar problems that share several constraints. We can use the commands push and pop for doing that. Each solver maintains a stack of assertions. The command push creates a new scope by saving the current stack size. The command pop removes any assertion performed between it and the matching push. The check method always operates on the content of solver assertion stack.

The following example shows an example that Z3 cannot solve. The solver returns unknown in this case. Recall that Z3 can solve nonlinear polynomial constraints, but 2**x is not a polynomial.

In [21]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
print (s.check())

unknown


The following example shows how to traverse the constraints asserted into a solver, and how to collect performance statistics for the check method.

In [23]:
x = Real('x')
y = Real('y')
s = Solver()
s.add(x > 1, y > 1, Or(x + y > 3, x - y < 2))
print ("asserted constraints...")
for c in s.assertions():
    print (c)

print (s.check())
print ("statistics for the last check method...")
print (s.statistics())
# Traversing statistics
for k, v in s.statistics():
    print ("%s : %s" % (k, v))

asserted constraints...
x > 1
y > 1
Or(x + y > 3, x - y < 2)
sat
statistics for the last check method...
(:add-rows     1
 :assert-lower 4
 :decisions    2
 :final-checks 1
 :max-memory   2.73
 :memory       2.37
 :mk-bool-var  9
 :num-allocs   130087780
 :pivots       1
 :rlimit-count 4339)
decisions : 2
final checks : 1
mk bool var : 5
add rows : 1
pivots : 1
assert lower : 4
mk bool var : 1
mk bool var : 1
mk bool var : 1
mk bool var : 1
num allocs : 130087780
rlimit count : 4339
max memory : 2.73
memory : 2.37


The command check returns sat when Z3 finds a solution for the set of asserted constraints. We say Z3 satisfied the set of constraints. We say the solution is a model for the set of asserted constraints. A model is an interpretation that makes each asserted constraint true. The following example shows the basic methods for inspecting models.

In [26]:
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
print (s.check())

m = s.model()
print ("x = %s" % m[x])

print ("traversing model...")
for d in m.decls():
    print ("%s = %s" % (d.name(), m[d]))
    
m.decls()

sat
x = 2
traversing model...
z = 0
y = 2
x = 2


[z, y, x]